# Tensorflow Guide

[From the TensorFlow Low Level Intro](https://www.tensorflow.org/guide/low_level_intro)

1. [Low Level Intro](#intro)
2. [Tensors](#tensors)
3. [Variables](#variables)
4. [Graphs and Sessions](#graphs)
5. [Save and Restore](#save)

In [36]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

import os

<a id='intro'></a>

---
---
---
# Introduction

## Tensors

Scalars, Vectors, Matrices, etc.

## TF Core

1. Building computational graph
    * Operations: consume/produce tensors (nodes)
    * Tensors: values that flow through graph (edges)
        * Don't have actual values, handle elements in graph.
2. Running the graph
    * tf.Session
    * Runs all the operations

In [2]:
# Example of building a simple graph 

a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


## TensorBoard

Visualizing a computation graph.

Run tensorboard --logdir . from the command line  

Visit the [graphs page](http://localhost:6006/#graphs) to see the current graph. 

In [3]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()

## Start a Session

Use the run method to run sessions. During a run, tf tensor assigned a single value.

Can handle multiple tensors, or tuples and dictionaries.

In [4]:
# Example sessions
sess = tf.Session()
print(sess.run(total))
print()

print(sess.run({'ab':(a, b), 'total':total}))
print()

vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print()
# New session resets the random numbers
print(sess.run(vec))
print()
# But stays consistent during a single run
print(sess.run((out1, out2)))

7.0

{'ab': (3.0, 4.0), 'total': 7.0}

[0.21971369 0.12934744 0.6974678 ]

[0.7522409  0.32861328 0.44966292]

(array([1.4715741, 1.2332177, 1.2504699], dtype=float32), array([2.471574 , 2.2332177, 2.25047  ], dtype=float32))


## Feeding

Can use placeholders instead of constants. 

* Use feed_dict argument to send in variable values.
* Can overwrite any tensor in the graph.

Can also use data instead of placeholders for more complex models.

* Need to make an iterator.
* Use make_one_shot_iterator method
* Might need to initialize iterator first if data depends on operations.

In [5]:
# Use placeholders to assign variables
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

7.5
[3. 7.]


In [6]:
# TF data
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

while True:
  try:
    print(sess.run(next_item))
  except tf.errors.OutOfRangeError:
    print('End of data')
    break
    
print()

    
# Initalize iterator first
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    print('End of data')
    break

[0 1]
[2 3]
[4 5]
[6 7]
End of data

[ 0.52168643  1.4267836  -1.079171  ]
[-0.0398622   1.45702    -0.31765458]
[-0.53401935 -0.7407698   0.888094  ]
[0.51265025 0.13362586 1.26846   ]
[ 0.81480944 -0.96578735  0.5240727 ]
[-0.22211717  0.01251148  0.00614682]
[ 0.17489392 -0.43458703 -1.1202222 ]
[ 1.4680849  -0.34176132 -1.169173  ]
[-0.74166256  0.4095481  -0.88163054]
[-2.399935   1.8603909 -0.7541033]
End of data


## Layers

Preferred way to add trainable parameters to a graph.

Package variables and operators that act on them. 

Must initialize variables before we can use them. 
* global_variables_initializer only initializes variables that existed in graph when initializer was created (create it last)

Some shortcuts exist: eg Dense shortcut is dense.
* Create and run layer in single cell.

In [7]:
# Creating Layers
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

# Initialize all variables and run graph.
init = tf.global_variables_initializer()
sess.run(init)

# Execute Layers
print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]}))

[[0.4131797]
 [0.7181771]]


In [8]:
# Shortcut method
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[-0.10449862]
 [-0.47251987]]


## Feature Columns

Can play with the columns if you want.

* feature_column.input_layer
* Need to wrap categorical columns in an indicator column

In [9]:
sess = tf.Session()

features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

# Remember to initialize variables (required for categorical)
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

# Run it
print(sess.run(inputs))


Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


## Training

1. Define data
2. Define model
3. Define loss
4. Define optimizer
    * Will automatically update variables to minimize the loss.
5. Run and see loss values

In [10]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(10):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

print(sess.run(y_pred))

4.787796
3.3509994
2.3538651
1.6618035
1.1814256
0.8479322
0.6163598
0.45550904
0.34373167
0.26600638
[[-0.63878775]
 [-1.5025278 ]
 [-2.366268  ]
 [-3.230008  ]]


<a id='tensors'></a>

---
---
---
# Tensors

Main computational element passed through grahphs. tf.Tensor has:
* data type: float32, int32, etc. Always known.
    * Can only ever have 1 type per tensor. (eg all strings, all ints, etc). 
    * Can cast into different datatypes
    * Converts python integers to tf.int32
    * Converts python floating point to tf.float32 (unless otherwise specified)
* shape: Can be partially unknown (eg. don't know number of examples to feed in). 
    * Can access either via the tf.Tensor.shape method or
    * tf.shape operation (ie tf.shape(tensor))
    * Can reshape like numpy arrays
* rank: number of dimensions
     * 0 = scalar
     * 1 = vector
     * 2 = matrix
     * ... etc

Special tensors:
* tf.Variable
* tf.constant
* tf.placeholder
* tf.SpareTensor

Can access slices the same way as regular python. 

In [11]:
# Rank 0 Examples
mammal = tf.Variable("Elephant", tf.string)
ignition = tf.Variable(451, tf.int16)
floating = tf.Variable(3.14159265359, tf.float64)
its_complicated = tf.Variable(12.3 - 4.85j, tf.complex64)

print('Rank 0')
print(ignition)
print(ignition.shape)
print(tf.rank(ignition))
print()

# Rank 1 Examples
mystr = tf.Variable(["Hello"], tf.string)
cool_numbers  = tf.Variable([3.14159, 2.71828], tf.float32)
first_primes = tf.Variable([2, 3, 5, 7, 11], tf.int32)
its_very_complicated = tf.Variable([12.3 - 4.85j, 7.5 - 6.23j], tf.complex64)

print('Rank 1')
print(mystr)
print(mystr.shape)
print(tf.rank(mystr))
print()

# Rank 2

mymat = tf.Variable([[7],[11]], tf.int16)
myxor = tf.Variable([[False, True],[True, False]], tf.bool)
linear_squares = tf.Variable([[4], [9], [16], [25]], tf.int32)
squarish_squares = tf.Variable([ [4, 9], [16, 25] ], tf.int32)
rank_of_squares = tf.rank(squarish_squares)
mymatC = tf.Variable([[7],[11]], tf.int32)

print('Rank 2')
print(mymat)
print(mymat.shape)
print(tf.rank(mymat))
print()

# And higher
# Images often rank 4 (batch x width x height x rgb)
my_image = tf.zeros([10, 299, 299, 3])  

print('Higher Rank')
print(my_image)
print(my_image.shape)
print(tf.rank(my_image))
print()

print("Run session to get actual rank: %d"%sess.run(tf.rank(my_image)))
print()

my_image.shape

Rank 0
<tf.Variable 'Variable_1:0' shape=() dtype=int32_ref>
()
Tensor("Rank:0", shape=(), dtype=int32)

Rank 1
<tf.Variable 'Variable_4:0' shape=(1,) dtype=string_ref>
(1,)
Tensor("Rank_1:0", shape=(), dtype=int32)

Rank 2
<tf.Variable 'Variable_8:0' shape=(2, 1) dtype=int32_ref>
(2, 1)
Tensor("Rank_3:0", shape=(), dtype=int32)

Higher Rank
Tensor("zeros:0", shape=(10, 299, 299, 3), dtype=float32)
(10, 299, 299, 3)
Tensor("Rank_4:0", shape=(), dtype=int32)

Run session to get actual rank: 4



TensorShape([Dimension(10), Dimension(299), Dimension(299), Dimension(3)])

In [12]:
# Reshaping a tensor
rank_three_tensor = tf.ones([3, 4, 5])
matrix = tf.reshape(rank_three_tensor, [6, 10])  # Reshape existing content into
                                                 # a 6x10 matrix
matrixB = tf.reshape(matrix, [3, -1])  #  Reshape existing content into a 3x20
                                       # matrix. -1 tells reshape to calculate
                                       # the size of this dimension.
matrixAlt = tf.reshape(matrixB, [4, 3, -1])  # Reshape existing content into a
                                             #4x3x5 tensor
    
print(rank_three_tensor.shape)
print(matrix.shape)
print(matrixB.shape)
print(matrixAlt.shape)

print(tf.shape(matrixAlt))

# Note that the number of elements of the reshaped Tensors has to match the
# original number of elements. Therefore, the following example generates an
# error because no possible value for the last dimension will match the number
# of elements.
# ERROR::: yet_another = tf.reshape(matrixAlt, [13, 2, -1])  # ERROR!

(3, 4, 5)
(6, 10)
(3, 20)
(4, 3, 5)
Tensor("Shape:0", shape=(3,), dtype=int32)


In [13]:
# Data Type Conversion 
# Cast a constant integer tensor into floating point.
float_tensor = tf.cast(tf.constant([1, 2, 3]), dtype=tf.float32)
print(tf.constant([1,2,3]).dtype)
print(float_tensor.dtype)

<dtype: 'int32'>
<dtype: 'float32'>


## Evaluating Tensors

Can fetch assigned values using the eval method. 

* only works when a default tf.Session is active
* may fail if dynamic information is not available

In [14]:
# Start a session and evaluate a tensor
sess = tf.Session()
constant = tf.constant([1, 2, 3])
tensor = constant * constant
print(tensor.eval(session=sess))

# Need to assign placeholder values
p = tf.placeholder(tf.float32)
t = p + 1.0

# t.eval()  # This will fail, since the placeholder did not get a value.
print(t.eval(session = sess, feed_dict={p:2.0}))  # This will succeed because we're feeding a value
                           # to the placeholder.

[1 4 9]
3.0


## Printing Tensors 

Don't use print, use tf.Print and its return value. 

In [15]:
# Print things
tf.Print(t, [t])  # This does nothing
t = tf.Print(t, [t])  # Here we are using the value returned by tf.Print
result = t + 1  # Now when result is evaluated the value of `t` will be printed.
print(result)
result.eval(session=sess, feed_dict={p:2.0}) # This will print result?

Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:
```python
    sess = tf.Session()
    with sess.as_default():
        tensor = tf.range(10)
        print_op = tf.print(tensor)
        with tf.control_dependencies([print_op]):
          out = tf.add(tensor, tensor)
        sess.run(out)
    ```
Additionally, to use tf.print in python 2.7, users must make sure to import
the following:

  `from __future__ import print_function`

Tensor("add_5:0", dtype=float32)


4.0

<a id='variables'></a>

---
---
---
# Variables

Best way to represent shared, persistant states manipulated by program. tf.Variable class. Exist outside the context of a session run (unlike tensors). 

## Create a Variable

tf.get_variable(name, shape)

defaults to tf.float32, uniform random initilization.
* Can be optionally specified
* Can't over-ride variables that have already been created without changing the tf.variable_scope

In [16]:
# Create a variable
# try/excepts just to 
try:
    my_variable = tf.get_variable("my_variable", [1, 2, 3])
    print(my_variable)
except:
    print(my_variable)

# Different initialization 
try:
    other_variable = tf.get_variable("other_variable", dtype=tf.int32,initializer=tf.constant([23, 42]))
    print(other_variable)
except:
    print(other_variable)

<tf.Variable 'my_variable:0' shape=(1, 2, 3) dtype=float32_ref>
<tf.Variable 'other_variable:0' shape=(2,) dtype=int32_ref>


## Variable Collections

It's possible to access all variables at once. All get placed into 2 collections:
* tf.GraphKeys.GLOBAL_VARIABLES
    * Shared across all devices
* tf.GraphKeys.TRAINABLE_VARIABLES
    * Trained on. IF you don't want it trained, add to LOCAL_VARIABLES
    
Can also create your own collections, use add_to_collection

In [17]:
print(tf.GraphKeys.GLOBAL_VARIABLES)
print(tf.GraphKeys.TRAINABLE_VARIABLES)
print(tf.GraphKeys.LOCAL_VARIABLES)

global_vars = tf.get_collection(
    tf.GraphKeys.GLOBAL_VARIABLES,
    scope=None
)

print(global_vars[-1])

variables
trainable_variables
local_variables
<tf.Variable 'other_variable:0' shape=(2,) dtype=int32_ref>


In [18]:
# Non trained variables
try:
    my_local = tf.get_variable("my_local", shape=(),
                    collections=[tf.GraphKeys.LOCAL_VARIABLES])
except:
    1
    
try:
    my_non_trainable = tf.get_variable("my_non_trainable",
                                   shape=(),
                                   trainable=False)
except: 
    1

trainable_vars = tf.get_collection(
    tf.GraphKeys.TRAINABLE_VARIABLES,
    scope=None
)

local_vars = tf.get_collection(
    tf.GraphKeys.LOCAL_VARIABLES,
    scope=None
)

print(trainable_vars[-1])
print(local_vars[-1])

<tf.Variable 'my_local:0' shape=() dtype=float32_ref>
<tf.Variable 'my_local:0' shape=() dtype=float32_ref>


In [19]:
# Create your own collection
tf.add_to_collection("my_collection_name", my_local)
tf.get_collection("my_collection_name")

[<tf.Variable 'my_local:0' shape=() dtype=float32_ref>]

## Initializing Variables

To do all in one go:
* tf.global_variables_initializer()
* random initialization to everything in tf.GraphKeys.GLOBAL_VARIABLES

To do yourself:
* variable.initializer

Can get errors if variable depends on other variables.
* Use variable.initialzed_value()

In [20]:
sess.run(tf.global_variables_initializer())
# Now all variables are initialized.

# Or do your own:
sess.run(my_variable.initializer)

# Check which hasn't been initalized
print(sess.run(tf.report_uninitialized_variables()))

# When being careful with making sure things are initialized in the right order
try:
    v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
    w = tf.get_variable("w", initializer=v.initialized_value() + 1)
except:
    1

[b'my_local']


In [21]:
sess.close()
tf.reset_default_graph()

## Using Variables

Treat like a normal tf.Tensor

To add value: use methods assign, assign_add, friends

TF optimizers have built in operations to efficiently update values of variables. 

In [23]:
# Just like a tensor
with tf.Session() as sess:
    try:
        v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
        w = v + 1  # w is a tf.Tensor which is computed based on the value of v.
               # Any time a variable is used in an expression it gets automatically
               # converted to a tf.Tensor representing its value.
    except: 
        1

    # Assigning values
    assignment = v.assign_add(1)
    tf.global_variables_initializer().run(session=sess)
    print(sess.run(assignment))  # or assignment.op.run(), or assignment.eval()

    # Checking value 
    assignment = v.assign_add(1)
    with tf.control_dependencies([assignment]):
        w = v.read_value()  # w is guaranteed to reflect v's value after the
        print(w)                  # assign_add operation.

1.0
Tensor("read:0", shape=(), dtype=float32)


## Sharing Variables

Two methods:
* Explicitly pass variables around
* Implicitly wrap within the variable_scope object 

Possible errors:
* Trying to overwrite variables name. 

In [ ]:
# Example convolutional layer
# Nice and clean/concise variable names (weights and biases)
def conv_relu(input, kernel_shape, bias_shape):
    # Create variable named "weights".
    weights = tf.get_variable("weights", kernel_shape,
        initializer=tf.random_normal_initializer())
    # Create variable named "biases".
    biases = tf.get_variable("biases", bias_shape,
        initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights,
        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

# however, fails in that tf will not know whether to reuse the weights/biases or not everytime conv_relu is called

# FIX: Call conv_relu within different scopes
def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        # Variables created here will be named "conv1/weights", "conv1/biases".
        relu1 = conv_relu(input_images, [5, 5, 32, 32], [32])
    with tf.variable_scope("conv2"):
        # Variables created here will be named "conv2/weights", "conv2/biases".
        return conv_relu(relu1, [5, 5, 32, 32], [32])
    
input1 = tf.zeros([5, 5, 32, 32])    
input2 = tf.zeros([5, 5, 32, 32])    

# If you wish to reuse variables: create a scope with the same name
with tf.variable_scope("model5"):
    output1 = my_image_filter(input1)
with tf.variable_scope("model5", reuse=True):
    output2 = my_image_filter(input2)
    
# Or create a scope using reuse_variable
with tf.variable_scope("model3") as scope:
    output1 = my_image_filter(input1)
    scope.reuse_variables()
    output2 = my_image_filter(input2)
    
# Or reuse a scope name
with tf.variable_scope("model4") as scope:
    output1 = my_image_filter(input1)
with tf.variable_scope(scope, reuse=True):
    output2 = my_image_filter(input2)

<a id="graphs"></a>

---
---
---
# Graphs and Sessions

1. Define dataflow graph
    * Nodes = computations
    * Edges = data
    * Parellelisable
    * Distributed execution
    * Compilation 
    * Portability

2. tf Graphs
    * Contain graph structure (nodes and edges)
    * Graph collections (collections of metadata)
    * Building:
        * Construct new tf.Operations (nodes) and tf.Tensor (edge) objects
        * Add them to a tf.Graph instance
    * EXAMPLE:
        * call tf.constant(42) creates an operation (produce 42)
        * adds to default graph
        * returns a tensor that represents the value
    * Calling tf.train.Optimizer.minimze 
        * Adds operations and tensors to default graph that will calculate gradiations
        * Returns an operation that will apply gradients
    * default graph
        * implicit argument to all API functions
   
3. Create TensorFlow session to run parts of graph across local/remote devices. 

## Naming Operations

tf.name_scope allows you to add name scope to make names that are more sensible than tf defaults.

In [ ]:
# Adding name scopes

c_0 = tf.constant(0, name="c")  # => operation named "c"

# Already-used names will be "uniquified".
c_1 = tf.constant(2, name="c")  # => operation named "c_1"

# Name scopes add a prefix to all operations created in the same context.
with tf.name_scope("outer"):
    c_2 = tf.constant(2, name="c")  # => operation named "outer/c"

    # Name scopes nest like paths in a hierarchical file system.
    with tf.name_scope("inner"):
        c_3 = tf.constant(3, name="c")  # => operation named "outer/inner/c"

    # Exiting a name scope context will return to the previous prefix.
    c_4 = tf.constant(4, name="c")  # => operation named "outer/c_1"

    # Already-used name scopes will be "uniquified".
    with tf.name_scope("inner"):
        c_5 = tf.constant(5, name="c")  # => operation named "outer/inner_1/c"

## Placing Operations on Devices

tf.device allows you to easily spread over devices

can automatically pin operations onto different machines

In [ ]:
# Operations created outside either context will run on the "best possible"
# device. For example, if you have a GPU and a CPU available, and the operation
# has a GPU implementation, TensorFlow will choose the GPU.
weights = tf.random_normal([2,2,1])

with tf.device("/device:CPU:0"):
  # Operations created in this context will be pinned to the CPU.
  img = tf.cast(tf.image.decode_png(tf.read_file("../Images/cnn.png")),tf.float32)

with tf.device("/device:GPU:0"):
  # Operations created in this context will be pinned to the GPU.
  result = tf.matmul(weights, img)

## Sessions

Connects tf.graph to the cpp runtime. 

Can either use with tf.Session() as sess: or open/close a session (like opening/closing a file). 

3 arguments:
* target (default is local machine)
* graph (default is default graph)
* config 

Run sessions with tf.Session.run
* must specify a list of fetches to determine return values (what parts of graph need to be run)

In [ ]:
# Create a default in-process session.
with tf.Session() as sess:
  # ...
    print('Do things')

# Create a remote session.
with tf.Session("grpc://example.org:2222"):
  # ...
    print('Do remote things')

In [ ]:
tf.reset_default_graph()

# Example run session
x = tf.constant([[37.0, -23.0], [1.0, 4.0]])
w = tf.Variable(tf.random_uniform([2, 2]))
y = tf.matmul(x, w)
output = tf.nn.softmax(y)
init_op = w.initializer

with tf.Session() as sess:
    # Run the initializer on `w`.
    sess.run(init_op)

    # Evaluate `output`. `sess.run(output)` will return a NumPy array containing
    # the result of the computation.
    print(sess.run(output))

    # Evaluate `y` and `output`. Note that `y` will only be computed once, and its
    # result used both to return `y_val` and as an input to the `tf.nn.softmax()`
    # op. Both `y_val` and `output_val` will be NumPy arrays.
    y_val, output_val = sess.run([y, output])
    
    
# Using dictionary of feeds
# Define a placeholder that expects a vector of three floating-point values,
# and a computation that depends on it.
x = tf.placeholder(tf.float32, shape=[3])
y = tf.square(x)

with tf.Session() as sess:
    # Feeding a value changes the result that is returned when you evaluate `y`.
    print(sess.run(y, {x: [1.0, 2.0, 3.0]}))  # => "[1.0, 4.0, 9.0]"
    print(sess.run(y, {x: [0.0, 0.0, 5.0]}))  # => "[0.0, 0.0, 25.0]"

    # Raises <a href="./../api_docs/python/tf/errors/InvalidArgumentError"><code>tf.errors.InvalidArgumentError</code></a>, because you must feed a value for
    # a `tf.placeholder()` when evaluating a tensor that depends on it.
    #sess.run(y)

    # Raises `ValueError`, because the shape of `37.0` does not match the shape
    # of placeholder `x`.
    #sess.run(y, {x: 37.0})

## Graph Visualization

Uses TensorBoard

from command line:
* tensorboard --logdir=/path/to/logdir
* then navigate to localhost:6006

In [ ]:
tf.reset_default_graph()

# Build your graph.
x = tf.constant([[37.0, -23.0], [1.0, 4.0]])
w = tf.Variable(tf.random_uniform([2, 2]))
y = tf.matmul(x, w)
# ...
loss = tf.losses.mean_squared_error(y,tf.matmul(x,w))
train_op = tf.train.AdagradOptimizer(0.01).minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # `sess.graph` provides access to the graph used in a <a href="./../api_docs/python/tf/Session"><code>tf.Session</code></a>.
    writer = tf.summary.FileWriter("/tmp/log/...", sess.graph)

    # Perform your computation...
    for i in range(1000):
        sess.run(train_op)
    # ...

    writer.close()

## Programming with Multiple Graphs

Multiple graphs can be named and used within a session

In [ ]:
tf.reset_default_graph()

g_1 = tf.Graph()
with g_1.as_default():
  # Operations created in this scope will be added to `g_1`.
  c = tf.constant("Node in g_1")

  # Sessions created in this scope will run operations from `g_1`.
  sess_1 = tf.Session()

g_2 = tf.Graph()
with g_2.as_default():
  # Operations created in this scope will be added to `g_2`.
  d = tf.constant("Node in g_2")

# Alternatively, you can pass a graph when constructing a <a href="./../api_docs/python/tf/Session"><code>tf.Session</code></a>:
# `sess_2` will run operations from `g_2`.
sess_2 = tf.Session(graph=g_2)

assert c.graph is g_1
assert sess_1.graph is g_1

assert d.graph is g_2
assert sess_2.graph is g_2

# Print all of the operations in the default graph.
g = tf.get_default_graph()
print(g.get_operations())

<a id='save'></a>

---
---
---
# Save and Restore

tf.train.Saver class. 

## Save and Restore Variables

TF Variables are best way to represent shared states. 
Saver object runs operations, saves checkpoint files, restores variables, etc.

Don't need to initialize restored variables. 

Can create as many Saver objects as you want/need to save and restore different subsets.

Saving variables:
* Saver passes all variables in the graph. 
* Uses name that was passed when it was created. 
* Can change these default settings to pass a subset/rename variables.
    * Use a lsit of variables or python dictionary

In [24]:
tf.reset_default_graph()

# Save variables
# Create some variables.
v1 = tf.get_variable("v1", shape=[3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", shape=[5], initializer = tf.zeros_initializer)

inc_v1 = v1.assign(v1+1)
dec_v2 = v2.assign(v2-1)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session() as sess:
    sess.run(init_op)
    # Do some work with the model.
    inc_v1.op.run()
    dec_v2.op.run()
    # Save the variables to disk.
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in path: %s" % save_path)

Model saved in path: /tmp/model.ckpt


In [25]:
# Restore Variables

tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("v1", shape=[3])
v2 = tf.get_variable("v2", shape=[5])

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "/tmp/model.ckpt")
    print("Model restored.")
    # Check the values of the variables
    print("v1 : %s" % v1.eval())
    print("v2 : %s" % v2.eval())

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
Model restored.
v1 : [1. 1. 1.]
v2 : [-1. -1. -1. -1. -1.]


In [26]:
# Save a subset

tf.reset_default_graph()
# Create some variables.
v1 = tf.get_variable("v1", [3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", [5], initializer = tf.zeros_initializer)

# Add ops to save and restore only `v2` using the name "v2"
saver = tf.train.Saver({"v2": v2})

# Use the saver object normally after that.
with tf.Session() as sess:
    # Initialize v1 since the saver will not.
    v1.initializer.run()
    saver.restore(sess, "/tmp/model.ckpt")

    print("v1 : %s" % v1.eval())
    print("v2 : %s" % v2.eval())

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
v1 : [0. 0. 0.]
v2 : [-1. -1. -1. -1. -1.]


## Inspect a Checkpoint

In [28]:
# import the inspect_checkpoint library
from tensorflow.python.tools import inspect_checkpoint as chkp


# print all tensors in checkpoint file
print('Print all tensors')
chkp.print_tensors_in_checkpoint_file("/tmp/model.ckpt", tensor_name='', all_tensors=True)
print()

# tensor_name:  v1
# [ 1.  1.  1.]
# tensor_name:  v2
# [-1. -1. -1. -1. -1.]

# print only tensor v1 in checkpoint file
print('Print only v1')
chkp.print_tensors_in_checkpoint_file("/tmp/model.ckpt", tensor_name='v1', all_tensors=False)
print()

# tensor_name:  v1
# [ 1.  1.  1.]

# print only tensor v2 in checkpoint file
print('Print only v2')
chkp.print_tensors_in_checkpoint_file("/tmp/model.ckpt", tensor_name='v2', all_tensors=False)

# tensor_name:  v2
# [-1. -1. -1. -1. -1.]

Print all tensors
tensor_name:  v1
[1. 1. 1.]
tensor_name:  v2
[-1. -1. -1. -1. -1.]

Print only v1
tensor_name:  v1
[1. 1. 1.]

Print only v2
tensor_name:  v2
[-1. -1. -1. -1. -1.]


## Save and Restore Models

Uses SavedModel instead of Saver.save()/restore() for variables. Language neutral.

Saves entire model:
* variables
* graphs
* metadata

Loading a Saved Model
* requires the session in which to restore and
* tags to identifty Meta-Graph stuff and
* location

Structure of SavedModel Directory:

assets/  
assets.extra/    
variables/  
    variables.data-?????-of-?????  
    variables.index  
saved_model.pb|saved_model.pbtxt

In [39]:
# Simple save
original = 'TF_Example_SaveModel'
count = 0
path_name = original+'_'+str(count)
while os.path.exists(path_name):
    path_name = original+'_'+str(count)
    count+=1
    
tf.reset_default_graph()
# Create some variables.
x = tf.get_variable("x", [5], initializer = tf.zeros_initializer)
y = tf.get_variable("y", [5], initializer = tf.zeros_initializer)
z = tf.add(x,y)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    tf.saved_model.simple_save(sess,
            path_name, # must be an empty or non-existant directory
            inputs={"x": x, "y": y},
            outputs={"z": z})

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: TF_Example_SaveModel_2\saved_model.pb


In [42]:
# Load a model
export_dir = path_name

with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], export_dir)

INFO:tensorflow:Restoring parameters from TF_Example_SaveModel_2\variables\variables


## SavedModel and Estimators

May want to build a model then create a service that takes requests, returns a result. 

Need to:

1. Specify output notes, corresponding APIs that can be served (classify, regress, predict).
2. Export model to SavedModel format
3. Serve model from local server and request predictons. 

## Command Line Interface

Can use a CLI to inspect and execute a saved model. 

Works from command line:

* Show tags:

usage: saved_model_cli show [-h] --dir DIR [--all] [--tag_set TAG_SET] [--signature_def SIGNATURE_DEF_KEY]
    
* run a graph computation

usage: saved_model_cli run [-h] --dir DIR 
                            --tag_set TAG_SET signature_def
                           SIGNATURE_DEF_KEY [--inputs INPUTS]
                           [--input_exprs INPUT_EXPRS]
                           [--input_examples INPUT_EXAMPLES] [--outdir OUTDIR]
                           [--overwrite] [--tf_debug]